In [2]:
#%pip install yfinance
#pip install sdmx1



In [1]:
#$env:PYTHONPATH="src"      # PowerShell
import sys
import os

project_root = os.path.abspath("../src")
sys.path.append(project_root)

%load_ext autoreload
%autoreload
from curves.yield_curve import YieldCurve
from products.coupon_bond import CouponBond
from products.future import EquityFuture
from products.swap import InterestRateSwap
from products.option import EuropeanCall,Option,EuropeanPut
from models.black_scholes import BlackScholesModel
from models.calibration import ImpliedVolatilitySolver
from data_loader.yield_curve_loader import YieldCurveLoader
from data_loader.equity_loader import EquityLoader

from models.monte_carlo import MonteCarloPricer



# les produits

In [ ]:
curve = YieldCurve(
    maturities=[1, 2, 3, 5],
    zero_rates=[0.02, 0.025, 0.03, 0.035]
)

print(curve.zero_rate(4))   # interpolation entre 3y et 5y
print(curve.discount_factor(4))
print(curve.zero_rate(0.5))



0.0325
0.8780954309205613
0.01


In [ ]:
bond = CouponBond(nominal=100, coupon_rate=0.05, maturity=5, payment_frequency=2)

print("Bond price =", bond.price(curve))
print("Bond price at t=1:", bond.price(curve,t=1))


Bond price = 106.98017032747649
Bond price at t=1: 109.0976296382205


In [ ]:
fut = EquityFuture(
    spot=4000,       # S&P 500 spot
    rate=0.03,       # 3%
    dividend_yield=0.015,  # 1.5%
    maturity=0.5     # 6 months
)

print("Future price =", fut.price())


Future price = 4030.1127817781353


In [ ]:
swap = InterestRateSwap(
    notional=1_000_000,
    fixed_rate=0.03,
    payment_times=[1, 2, 3]
)

print("Fixed Leg PV =", swap.pv_fixed_leg(curve))
print("Floating Leg PV =", swap.pv_floating_leg(curve))
print("Swap Value =", swap.price(curve))
print("Swap rate =", swap.swap_rate(curve))

print("\nFixed Leg PV =", swap.pv_fixed_leg(curve,t=1))
print("Floating Leg PV =", swap.pv_floating_leg(curve,t=1))
print("Swap Value =", swap.price(curve,t=1))
print("Swap rate =", swap.swap_rate(curve,t=1))

Fixed Leg PV = 85360.77849236093
Floating Leg PV = 66267.48803552707
Swap Value = -19093.290456833856
Swap rate = 0.02328967326889742

Fixed Leg PV = 87942.84293422407
Floating Leg PV = 48770.57549928599
Swap Value = -39172.26743493808
Swap rate = 0.017140392705175672


In [ ]:
call = EuropeanCall(strike=100, maturity=1.0)
model = BlackScholesModel(spot=105, rate=0.03,vol = 0.2)



print("Call price =", model.price(call))
print("Delta =", model.delta(call))
print("Gamma =", model.gamma(call))
print("Vega =", model.vega(call))


Call price = 12.638755916496294
Delta = 0.6893295440636719
Gamma = 0.01681549194942603
Vega = 37.078159748484396


In [ ]:
model2 = MonteCarloPricer(spot=105, rate=0.03, vol = 0.2)



print("Call price =", model2.price(call))
print("Delta =", model2.delta(call))
print("Gamma =", model2.gamma(call))
print("Vega =", model2.vega(call))

Call price = 12.525527486817198
Delta = 1902.5154354270855
Gamma = 86115997.25625342
Vega = 3525.7162977064827


In [ ]:
import streamlit as st

In [ ]:
# Market price of the call
market_price = 12.50

# Solver
solver = ImpliedVolatilitySolver(model)

iv = solver.implied_vol_call(call, market_price)

print("Implied volatility =", iv)

Implied volatility = 0.19625254622411945


# data

In [2]:
YC = YieldCurveLoader()
df = YC.get_rate(key='SR_20Y')
#YC.reload_yield_curve()
df

In [3]:
EQ = EquityLoader()
data = EQ.find('cac 40')
data.head()

,Symbole,Nom,Dernier cours,Secteur/Catégorie,Type,Change
0,^FCHI,CAC 40,"8 122,71",--,Indices,PAR
1,GC40.DE,AIS-AMUNDI CAC 40 ESG N,"142,96",--,Trackers,GER
2,CAC6L.PA,CAC 40 X6 Leverage NR,"3 102,46",--,Indices,PAR
3,LVC.PA,Amundi CAC 40 Daily (2x) Levera,"43,15",--,Trackers,PAR
4,CA10S.PA,CAC 40 X10 Short GR,"2 784,22",--,Indices,PAR


In [5]:
data = EQ.get_historic('^FCHI')
data.head()

e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, end=end, period=period)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^FCHI']: SSLError('Failed to perform, curl: (77) error setting certificate verify locations:  CAfile: e:\\Cours3A\\Modèles de pricing des derivatives\\asset-pricing\\venv\\Lib\\site-packages\\certifi\\cacert.pem CApath: none. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^FCHI,^FCHI,^FCHI,^FCHI,^FCHI,^FCHI
Date,,,,,,


In [ ]:
EQ = EquityLoader()
print(f' VIX : {EQ.get_VIX()}')
print(f' VCAC : {EQ.get_VCAC()}')

e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, end=end, period=period)
[*********************100%***********************]  1 of 1 completed
e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:96: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data.iloc[-1]['Close']) / 100


 VIX : 0.18559999465942384
 VCAC : 1.0000000000000003e-05


In [ ]:
EQ.get_implied_volatility('^FCHI')

np.float64(1.0000000000000003e-05)

In [ ]:
EQ.get_implied_volatility('^FCHI')

9

# Other